In [34]:
import h5py
import numpy as np
import nrrd
import os
from helpers import *

In [28]:
# Load the data
current_directory = os.getcwd()
base_pred_path = f'{current_directory}/data/Vesuvius/test/predictions/'
filename = '3350_4000_8450_xyz_256_res1_s4'
filename = 'manual_1'
pred_path = f'{base_pred_path}{filename}_predictions.h5'
pred_data = read_hdf5_file(pred_path)

img_path = f'{current_directory}/data/Vesuvius/test/raw/{filename}_raw.nrrd'
img, _ = nrrd.read(img_path)

pred = pred_data['predictions']

gt_label_data = read_hdf5_file(f'{current_directory}/data/Vesuvius/train/dataset/{filename}.h5') #may need to remove/add _raw if using layers_X or not
gt = gt_label_data['label']

Datasets in the file:
predictions
predictions shape: (256, 256, 256)


In [30]:
#show prediction
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(pred[:,slice_index,:])
    elif axis == 2:
        plt.imshow(pred[:,:,slice_index])
    else:
        plt.imshow(pred[slice_index,:,:])
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [17]:
#show ground truth label
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(gt[:,slice_index,:])
    elif axis == 2:
        plt.imshow(gt[:,:,slice_index])
    else:
        plt.imshow(gt[slice_index,:,:])
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [33]:
#show prediction overlaid on raw data
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(mark_boundaries_color(img[:,slice_index,:], pred[:,slice_index,:]))
    elif axis == 2:
        plt.imshow(mark_boundaries_color(img[:,:,slice_index], pred[:,:,slice_index]))
    else:
        plt.imshow(mark_boundaries_color(img[slice_index,:,:], pred[slice_index,:,:]))
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>

In [31]:
labeled_arr = label_foreground_structures(pred)

Number of connected foreground structures before filtering: 5
Number of connected foreground structures after filtering: 5


In [32]:
#show post processing instance segmentation overlaid on raw data
def plot_slice(slice_index, axis=0):
    plt.figure(figsize=(8, 6))
    if axis == 1:
        plt.imshow(mark_boundaries_color(img[:,slice_index,:], labeled_arr[:,slice_index,:]))
    elif axis == 2:
        plt.imshow(mark_boundaries_color(img[:,:,slice_index], labeled_arr[:,:,slice_index]))
    else:
        plt.imshow(mark_boundaries_color(img[slice_index,:,:], labeled_arr[slice_index,:,:]))
    plt.colorbar()
    plt.title(f'Slice {slice_index}')
    plt.show()

interact(plot_slice, slice_index=IntSlider(min=0, max=pred.shape[0]-1, step=1, value=0), axis=IntSlider(min=0, max=2, step=1, value=0))

interactive(children=(IntSlider(value=0, description='slice_index', max=255), IntSlider(value=0, description='…

<function __main__.plot_slice(slice_index, axis=0)>